In [1]:
import syft as sy
import torch
import pandas as pd

In [2]:
duet = sy.launch_duet(loopback=True)
#duet=sy.launch_duet(network_url="http://ec2-3-15-165-254.us-east-2.compute.amazonaws.com:5000/")

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...


/home/iwan/anaconda3/envs/pysft_tutorial/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/home/iwan/anaconda3/envs/pysft_tutorial/lib/python3.9/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!



In [3]:
df_all = pd.read_csv('data/train1.csv')

In [4]:
features = df_all.columns[2:]
target = 'label'

In [5]:
df_all.sample(5)

,label,filename,0,1,2,3,4,5,6,7,...,490,491,492,493,494,495,496,497,498,499
1825,3,Data/12k_DE/OR007@3_1.mat,-0.024722,-0.613748,-0.071173,0.495824,0.127084,-0.268473,-0.029032,0.204663,...,-0.416927,-1.705600,0.185508,1.900387,0.632305,-1.549963,-1.501117,0.671095,1.510098,-0.380053
5328,1,Data/12k_DE/B021_3.mat,0.405863,-1.065807,-1.876052,-0.036465,1.480678,0.210192,0.101332,2.033243,...,1.313944,-0.192175,-0.649660,0.263932,-0.205955,-0.826040,-0.229380,1.815524,1.196817,-0.002015
6969,1,Data/12k_DE/B028_1.mat,0.969299,-1.442636,-0.352134,1.913867,-0.109978,-2.412462,0.723535,2.876075,...,-0.104365,0.238422,0.016713,-0.003333,0.049989,-0.160895,-0.049038,0.435274,-0.144056,-0.596695
6131,3,Data/12k_DE/OR021@3_2.mat,-0.049538,0.135715,0.122316,-0.217896,-0.066432,0.317473,-0.206828,-0.631511,...,0.299996,-0.549371,-1.085905,0.148531,0.240575,-2.256261,-2.040133,0.740409,-0.378099,-1.853132
6660,1,Data/12k_DE/B007_3.mat,0.350244,1.564456,-0.985600,-1.685781,1.260903,1.625801,-0.989831,-0.843872,...,0.035057,-0.712720,-0.897813,-0.360514,0.038230,-0.179651,-1.257423,-0.858679,1.100136,0.607260


In [6]:
df_all.shape

(8171, 502)

In [7]:
df_all.shape[0]

8171

In [8]:
X = torch.tensor(df_all[features].values, dtype=torch.float32)
y = torch.tensor(df_all[target].values, dtype=torch.long)

In [9]:
print(X.shape)

torch.Size([8171, 500])


In [10]:
ds = [t for t in zip(X, y)]
ds = sy.lib.python.List(ds)

In [11]:
X = X.tag("bearing-data")
y = y.tag("bearing-target")
ds = ds.tag("bearing-tensordata")

X = X.describe(
    f"This is a dataset for bearing fault classification of {df_all.shape[0]} samples. 500 Features are machine sequence"
)
y = y.describe("Labels for bearing fault classification: Normal (N), Fault at Ball (B), Fault at Inner Raceway (IR), Fault at Outer Raceway (OR)")
ds = ds.describe("Dataset wrapper for both target and input")

In [12]:
data_pointer = X.send(duet, pointable=True)
target_pointer = y.send(duet, pointable=True)

In [13]:
ds = ds.send(duet, pointable=True)

In [14]:
duet.store

[<syft.proxy.torch.TensorPointer object at 0x7f94f6e72760>, <syft.proxy.torch.TensorPointer object at 0x7f94f6e720d0>, <syft.proxy.syft.lib.python.ListPointer object at 0x7f94f6e72310>]

In [15]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: c3199536a1fe41079b575530ea59d548>,[bearing-data],This is a dataset for bearing fault classifica...,<class 'torch.Tensor'>
1,<UID: 41a6cd0847c4444c8a188277b920eece>,[bearing-target],Labels for bearing fault classification: Norma...,<class 'torch.Tensor'>
2,<UID: c286fb47f0f74da7b41a2a2a76078627>,[bearing-tensordata],Dataset wrapper for both target and input,<class 'syft.lib.python.list.List'>


# Request Access

In [16]:
duet.requests.add_handler(
    action="accept",
    print_local=True
)

[2021-12-30T17:51:48.934044+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 8c65223724de489facb9cf8600c48d84>: To run test and inference locally
Value: <Storable: True>
[2021-12-30T17:52:03.482769+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 12fa314ccdab46c1b69a15b3771ab09b>:
Value: <Storable: 8171>
[2021-12-30T17:53:13.099016+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 31bf85163e894d818fc2732bb9b6e098>: To run test and inference locally
Value: <Storable: True>
[2021-12-30T17:53:27.726959+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 288eff139e614a898e7788d776e235b0>:
Value: <Storable: 8171>
[2021-12-30T17:53:34.040930+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: eddd7e18554d45099ab0d3be331b015b>:
Value: <Storable: 128>
[2021-12-30T17:53:34.943023+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 1269bb7d82fe4079890240ef3b0fc190>: To evaluate training progress
Value: <Storable: 1.432813048362732>
[2021-12-30T17:53:36.838404+0900

[2021-12-30T17:54:03.516852+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: c10d78d524ac4fc7b72576449f784120>: To evaluate training progress
Value: <Storable: '64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True'>
[2021-12-30T17:54:03.668052+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: aa6f70ddb02e42c28d2f2430de241e47>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([1.0398, 0.9574, 0.9817, 1.0780, 0.9167, 1.2363, 0.8472, 0.9546, 0.9909,    0.9837, 0.8745, 1.0248, 0.8479, 0.9885, 0.9143, 0.8078, 1.0234, 1.2271,    0.7623, 0.8535, 1.0148, 0.9278, 0.9787, 0.9153, 0.9466, 0.9478, 1.1634,    1.1184, 0.8847, 0.8294, 1.1511, 0.9904, 1.1284, 0.9450, 0.8880, 1.1171,    0.9089, 0.9367, 1.1314, 1.0718, 1.0493, 1.0084, 0.9450, 0.8640, 0.9348,    0.8163, 0.9486, 1.0842, 1.0036, 0.9097, 0.9355, 0.8985, 0.8418, 0.9225,    1.0220, 0.8675, 0.8551, 0.8846, 1.0927, 0.9229, 1.0721, 0.8694, 0.9838,    0.9134]

[2021-12-30T17:54:12.668792+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 81547f236a0d46deb79d6ed597ea746b>: To evaluate training progress
Value: <Storable: 'p=0.5, inplace=False'>
[2021-12-30T17:54:13.781194+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 2161f1310f0945948b4283f95f583b9e>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:54:14.884423+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: f0b3a214620b4ebbac3050977a53dc93>: To evaluate training progress
Value: <Storable: 'kernel_size=(2,), stride=(2,), padding=(0,)'>
[2021-12-30T17:54:15.034507+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: f0a9216332274e068f04daa102323dde>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:54:16.135629+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 892d71412f9e4645a51690b5e915f5ee>: To evaluate training progress
Value: <Storable: 'in_features=16000, out_features=4, bias=True'>
[2021-12-30T17:5

[2021-12-30T17:55:30.674005+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 65dde22a1ec14e6c921ceeaafaa74e87>: To evaluate training progress
Value: <Storable: '64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True'>
[2021-12-30T17:55:31.800800+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 628f4852fa0c49a2a1052d5ce486e162>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([1.0607, 0.8386, 0.9934, 1.0387, 0.9262, 1.2786, 0.9527, 0.9911, 0.9786,    0.9101, 0.7749, 1.0198, 0.7317, 0.9175, 0.6926, 0.8896, 0.9140, 1.1504,    0.8347, 0.8444, 0.9620, 0.7885, 0.7239, 0.8649, 0.9235, 0.9918, 1.2418,    1.1022, 0.8051, 0.7808, 1.0690, 0.9266, 1.1837, 0.9124, 0.9064, 1.1819,    0.7945, 0.8968, 1.1822, 1.0520, 1.0822, 0.9628, 0.9452, 0.8824, 0.8925,    0.7411, 0.8472, 1.0921, 0.9491, 0.7081, 0.9073, 0.8792, 0.6724, 0.9994,    0.9340, 0.7587, 0.7515, 0.8254, 1.0490, 0.9103, 1.1414, 0.7641, 0.8987,    0.8643]

[2021-12-30T17:55:41.772374+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 6540998208024ea983503b88928f33da>: To evaluate training progress
Value: <Storable: 'p=0.5, inplace=False'>
[2021-12-30T17:55:42.875625+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 77a2e1b0af874949bcc02df164683e40>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:55:43.973088+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 10abcafbd18e41228221d1ddbb8efabf>: To evaluate training progress
Value: <Storable: 'kernel_size=(2,), stride=(2,), padding=(0,)'>
[2021-12-30T17:55:44.099572+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 1479b85a8aa7447396eca8524c5cad34>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:55:45.203122+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 64c409f10066437985fb4fd77e493515>: To evaluate training progress
Value: <Storable: 'in_features=16000, out_features=4, bias=True'>
[2021-12-30T17:5

...

[2021-12-30T17:58:23.118334+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 10022469b6484758a645e497cb81b58f>: To evaluate training progress
Value: <Storable: '64, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True'>
[2021-12-30T17:58:24.224102+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 02c9b44a70ea4df99af8cac76c0e2ddb>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([1.1222, 0.7923, 1.0218, 0.9988, 0.9045, 1.2077, 0.9985, 1.0138, 1.0096,    0.9121, 0.6386, 0.9357, 0.7561, 0.8497, 0.7183, 0.9220, 0.8823, 1.1717,    0.9135, 0.7690, 0.9008, 0.8325, 0.6560, 0.8153, 0.8584, 1.0876, 1.2745,    1.0944, 0.7816, 0.8690, 1.1414, 0.8383, 1.1917, 0.7911, 0.9567, 1.2156,    0.7098, 0.8788, 1.1665, 0.9866, 1.0504, 0.8546, 0.9295, 0.8664, 0.8867,    0.7774, 0.7963, 1.0965, 0.9730, 0.5316, 0.9372, 0.8931, 0.6649, 1.0802,    0.8626, 0.6105, 0.7937, 0.8357, 1.1428, 0.9389, 1.1237, 0.7221, 0.8938,    0.7708]

[2021-12-30T17:58:31.021894+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: bec97d56c75b4b37ad69d0618d3b738c>: To evaluate training progress
Value: <Storable: '128, eps=9.999999747378752e-06, momentum=0.10000000149011612, affine=True, track_running_stats=True'>
[2021-12-30T17:58:31.191824+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 1307cd1ddff249a593d8c77f76259c28>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([0.5962, 0.5345, 0.6091, 0.4954, 0.5928, 0.7347, 0.6306, 0.4440, 0.6765,    0.4995, 0.6871, 0.4725, 0.4875, 0.6256, 0.7362, 0.6049, 0.4356, 0.5465,    0.5260, 0.6247, 0.5506, 0.7024, 0.5308, 0.4637, 0.4476, 0.5004, 0.5386,    0.4874, 0.7246, 0.4179, 0.4774, 0.4089, 0.5195, 0.5437, 0.5854, 0.5526,    0.5051, 0.5466, 0.5944, 0.4816, 0.4399, 0.4975, 0.6661, 0.6368, 0.5517,    0.6343, 0.7279, 0.6896, 0.5372, 0.4849, 0.5859, 0.4114, 0.6654, 0.7106,    0.4901, 0.5935, 0.6294, 0.7343, 0.5151, 0.6959, 0.4519, 0.6920, 0.4948,    0.5058

[2021-12-30T17:59:24.997573+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 1232543b555f43b1b2ea352637fc3fa1>: To evaluate training progress
Value: <Storable: 0.11838511377573013>
[2021-12-30T17:59:26.348495+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: bb3870da395a41fba35ea76fb573eaa9>: To evaluate training progress
Value: <Storable: 0.03382381424307823>
[2021-12-30T17:59:27.720120+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 163ddbff8c55494a8372b57e35b73e2d>: To evaluate training progress
Value: <Storable: 0.0016504438826814294>
[2021-12-30T17:59:29.065041+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: bd9eab19427f48bbb24333ec82e1049c>: To evaluate training progress
Value: <Storable: 0.0740840882062912>
[2021-12-30T17:59:30.417012+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 68cd12da11c844b4b9c2d46ed9d43ef1>: To evaluate training progress
Value: <Storable: 0.016956264153122902>
[2021-12-30T17:59:31.653215+0900][CRITICAL][logger]][25038

[2021-12-30T17:59:46.408477+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: a25807a3ab1c4b4b8fdb937dc705beaa>: To evaluate training progress
Value: <Storable: OrderedDict([('weight', tensor([1.0928, 0.7495, 0.9620, 0.9821, 0.8350, 1.1987, 0.9872, 1.0542, 1.0176,    0.8911, 0.5685, 0.9682, 0.7435, 0.8266, 0.7764, 0.9256, 0.9164, 1.1789,    0.9315, 0.7768, 0.8592, 0.8874, 0.5976, 0.8865, 0.8791, 1.1910, 1.3604,    1.1212, 0.7767, 0.8563, 1.0592, 0.7297, 1.1690, 0.7295, 0.9195, 1.2015,    0.6888, 0.8507, 1.1754, 1.1023, 1.0202, 0.8209, 0.9428, 0.8891, 0.9028,    0.6675, 0.7695, 1.0792, 0.9605, 0.5715, 0.9757, 0.9033, 0.6551, 1.0992,    0.8727, 0.6280, 0.8017, 0.8290, 1.1588, 0.9530, 1.1702, 0.6378, 0.8590,    0.6792])), ('bias', tensor([-0.2120, -0.3691, -0.1222, -0.3919, -0.3353, -0.2242, -0.2793, -0.4007,    -0.1198, -0.2488, -0.4880, -0.0350, -0.4107, -0.3413, -0.3550, -0.2896,    -0.7245, -0.4940, -0.0174, -0.4713, -0.2515, -0.5875, -0.6328, -0.5641,    -0.3450, -0.0449, -0.31

[2021-12-30T17:59:54.434113+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 2becd684f09641bb99502b7368f39b4f>: To evaluate training progress
Value: <Storable: 'p=0.5, inplace=False'>
[2021-12-30T17:59:55.538332+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 389be0fe6f5e4443a7951d6b808c1fd4>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:59:55.664318+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 1b4a82fc75134aeea054851e2a3bc9bc>: To evaluate training progress
Value: <Storable: 'kernel_size=(2,), stride=(2,), padding=(0,)'>
[2021-12-30T17:59:56.780681+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 3a0cf2f6a7e44a17b5ca9c7ed36b0a8b>: To evaluate training progress
Value: <Storable: OrderedDict()>
[2021-12-30T17:59:57.876691+0900][CRITICAL][logger]][25038] > HANDLER Request <UID: 945bb402ad6c40928f361d4a074fbe00>: To evaluate training progress
Value: <Storable: 'in_features=16000, out_features=4, bias=True'>
[2021-12-30T17:5